In [1]:
import pandas as pd

# Load the dataset into a Pandas dataframe
ratings = pd.read_csv('movies.csv')

ModuleNotFoundError: No module named 'pandas'

In [13]:
# Check for missing values
print(ratings.isnull().sum())

# Check for duplicates
print(ratings.duplicated().sum())

# Remove duplicates
ratings.drop_duplicates(inplace=True)

movieId    0
title      0
genres     0
rating     0
dtype: int64
0


In [14]:
# Convert the dataframe into a pivot table
user_item_matrix = pd.pivot_table(ratings, index='movieId', columns='title', values='rating', fill_value=0)

# Convert the pivot table to a sparse matrix
from scipy.sparse import csr_matrix
user_item_matrix = csr_matrix(user_item_matrix.values)

In [15]:
from sklearn.neighbors import NearestNeighbors

# Train the KNN model using the sparse matrix
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(user_item_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [16]:
# Function to get movie recommendations for a user
def get_movie_recommendations(movie_id, num_recommendations=5):
    # Find the k nearest neighbors of the movie
    distances, indices = knn_model.kneighbors(user_item_matrix[movie_id], n_neighbors=num_recommendations+1)

    # Get the indices of the recommended movies
    recommended_movie_indices = indices.squeeze()[1:]

    # Get the titles of the recommended movies
    recommended_movie_titles = user_item_matrix[recommended_movie_indices].transpose().index

    return recommended_movie_titles

In [20]:
# Function to get movie recommendations for a movie
def get_movie_recommendations(movie_id, num_recommendations=5):
    # Find the k nearest neighbors of the movie
    distances, indices = knn_model.kneighbors(user_item_matrix[movie_id], n_neighbors=num_recommendations+1)

    # Get the indices of the recommended movies
    recommended_movie_indices = indices.squeeze()[1:]

    # Convert the sparse matrix to a dense matrix
    user_item_dense = user_item_matrix.toarray()

    # Get the titles of the recommended movies
    recommended_movie_titles = pd.DataFrame(user_item_dense).iloc[recommended_movie_indices].index

    return recommended_movie_titles

# Test the model by getting recommendations for a movie
movie_id = 42
recommended_movies = get_movie_recommendations(movie_id)
print(recommended_movies)

Int64Index([6884, 6887, 6886, 6889, 6883], dtype='int64')


In [24]:
recommended_movie_indices = [6884, 6887, 6886, 6889, 6883]
recommended_movie_titles = ratings.loc[recommended_movie_indices, 'title']
print(recommended_movie_titles)

6884    Lady Vengeance (Sympathy for Lady Vengeance) (...
6887                     District 13 (Banlieue 13) (2004)
6886                                     Ted Bundy (2002)
6889                                        Hostel (2005)
6883    We All Loved Each Other So Much (C'eravamo tan...
Name: title, dtype: object


In [42]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load the dataset into a Pandas dataframe
ratings = pd.read_csv('movies.csv')

# Check for missing values
print(ratings.isnull().sum())

# Check for duplicates
print(ratings.duplicated().sum())

# Remove duplicates
ratings.drop_duplicates(inplace=True)

# Convert the dataframe into a pivot table
user_item_matrix = pd.pivot_table(ratings, index=ratings.index, columns='movieId', values='rating', fill_value=0)

# Convert the pivot table to a sparse matrix
user_item_matrix = csr_matrix(user_item_matrix.values)

# Train the KNN model using the sparse matrix
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(user_item_matrix)

# Function to get movie recommendations for a user
def get_movie_recommendations(movie_id, num_recommendations=5):
    # Find the k nearest neighbors of the movie
    distances, indices = knn_model.kneighbors(user_item_matrix[movie_id], n_neighbors=num_recommendations+1)

    # Get the indices of the recommended movies
    recommended_movie_indices = indices.squeeze()[1:]

    # Get the titles of the recommended movies
    #recommended_movie_titles = user_item_matrix[recommended_movie_indices].transpose().index

    return recommended_movie_titles

# Test the model by getting recommendations for a movie
movie_id = 42
recommended_movies = get_movie_recommendations(movie_id)
print(recommended_movies)

# Assume the following are the actual ratings for the recommended movies by a user
actual_ratings = [4, 3, 5, 4, 3]

# Calculate the mean absolute error and root mean squared error
predicted_ratings = [ratings.loc[ratings['title'] == title, 'rating'].mean() for title in recommended_movies]
mae = mean_absolute_error(actual_ratings, predicted_ratings)
rmse = mean_squared_error(actual_ratings, predicted_ratings, squared=False)

print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')


movieId    0
title      0
genres     0
rating     0
dtype: int64
0
6884    Lady Vengeance (Sympathy for Lady Vengeance) (...
6887                     District 13 (Banlieue 13) (2004)
6886                                     Ted Bundy (2002)
6889                                        Hostel (2005)
6883    We All Loved Each Other So Much (C'eravamo tan...
Name: title, dtype: object
Mean Absolute Error: 3.8
Root Mean Squared Error: 3.872983346207417
